In [2]:
import json, sys, random, copy, sys
import random
import luigi



In [1]:
import json, sys, random, copy, sys
import random

path = "../outils"
if path not in sys.path :
    sys.path.append (path)
    
from gestionDate import gestionDate



path = "../outils"
if path not in sys.path :
    sys.path.append (path)
from gestionDate import gestionDate

# on load les produits, clients, lienClientProduit

nomFile = "../data/generation/dico_client.json"
f = open (nomFile , 'r')
dico_client = json.loads(f.read ())
f.close()

nomFile = "../data/generation/produit.json"
f = open (nomFile , 'r')
dicoProduit = json.loads(f.read ())
f.close()

nomFile = "../data/generation/lienClientProduit.json"
f = open (nomFile , 'r')
dicoLien = json.loads(f.read ())
f.close()


# class ecriture evenement
#pathEvenements = "../data/generation/evenements.txt"


class ecritureEvenements () :
    def __init__ (self,f_evenement = None,
                  isTest = False) :
        self.isTest = isTest
        self.f = f_evenement
        
            
            
            
        
    def ecritureEvenement (self, IDstring, dateString, produitChaine, prixPanierString) :
        # tous les elements sont en chaine !!!!!
        ligne = [IDstring, dateString, prixPanierString] + produitChaine
        ligne = '|'.join (ligne)
        if self.isTest :
            return print (ligne)
        return self.f.write (ligne + '\n')
    
    

class ecriturePaniers () :
    def __init__ (self,
                  typePas,
                  f_evenement = None,
                  pathProduit = "../data/generation/produit.json",
                  pathLienClientProduit = "../data/generation/lienClientProduit.json",
                  isTest = False ) :
        
        self.typePas = typePas
        self.f_evenement = f_evenement
               
        f = open (pathProduit , 'r')
        self.dicoProduit = json.loads(f.read ())
        f.close()
        
        f = open (pathLienClientProduit , 'r')
        self.LienClientProduit = json.loads(f.read ())
        f.close()
        
        E = ecritureEvenements  ( f_evenement = f_evenement,
                                  isTest = isTest)
        self.ecritureEvenement = E.ecritureEvenement
        self.isTest = isTest
        
        
    def ecriturePanier (self, ID, 
                              dateString,
                              nombreMinAchat,                        
                              nombreMaxAchat,
                        isTest = False,
                        ) :
        """
        ID exemple :
                    'homme_cadre_sup_0'
                    'homme_cadre_0'
        
        dateString : format standart ex "2021-01-20 16:59:17"
        
        genere un certain nombre d'achat dispersion uniforme entre nombreMin et nombreMax
        
        """
        horloge = gestionDate (self.typePas, dateDebut = dateString  ) # pour generer des date d'achat credible ?
        t = ID.split('_') [:-1]
        typeClient = '_'.join(t)
        
        listeTypeProduit= self.LienClientProduit [typeClient]
        
        # on fabrique la liste completes des produits possibles
        listeProduitPossible = []
        for typeProduit in listeTypeProduit :
            listeProduit = self.dicoProduit [typeProduit]
            listeProduitPossible.extend(listeProduit)
            continue
        
        
        
        # on agrandie artificielle la liste possible en fonction de nombreMax - nombreMin
        listeCalcul = copy.copy(listeProduitPossible)
        repetition = nombreMaxAchat-nombreMinAchat 
        for _ in range (0, repetition ):
            listeCalcul.extend(listeProduitPossible)
            continue
            
        listeProduitPossible = listeCalcul
        random.shuffle (listeProduitPossible) # deprecated a refaire 
             
        # on calcul le nombre d'achat dispersion uniforme entre nombreMin et nombreMax
        
        choix = [i for i in range(nombreMinAchat, nombreMaxAchat ) ]
        if isTest :
            print ("choix =", choix, " et ",nombreMinAchat, nombreMaxAchat )
           
        nombreAchat = random.choice (choix)
        if isTest :
            print ("choix = ", choix, "\n")
            print ("nombreAchat = ", nombreAchat, "\n")
        
        # on achete tous les produits dans la sous liste
        sousListe = listeProduitPossible [0 : nombreAchat]
        if isTest :        
            print ("sousListe = ", sousListe, "\n")
            print ("nombreAchat = ", nombreAchat, "\n")
        
        ####  pour ensuite faire appel ecriture param : IDstring, dateString, produitChaine = liste
        # calcul la position du prix (toujours à la fin de la liste produit)
        positionPrix = len(sousListe [0]) - 1
        if isTest :
            print ("sousListe = ", sousListe, "\n")
            print ("positionPrix = ", positionPrix, "\n")
            print ('\n')
        # prix panier ?
        prixPanier = 0.
        try:
            for produit in sousListe :
                prixPanier += float (produit [positionPrix])
                continue
        except :
            print ("produit = ", produit, "\n")
            raise
        
        # on ecrit la liste des achats avec le prix panier
        prix = (prixPanier*100.)
        prix = int(prix) / 100.
        
        prixPanier = str (prix)
        
        for produit in sousListe :
            date = horloge.aleaDate ()
            self.ecritureEvenement(ID, date, produit, prixPanier)
            continue
            
        return prixPanier
    
    
    
    
    
class Generation  () :
    def __init__ (self,
                  dateDebutGeneral,
                  f_evenement = None,
                  typePas = 0, # demi journee (en general voir pls tard l'heure)
                  typeFrequence = 2, # semaine (en general voir plus tard si necessaire
                  
                  # si nouveau contexte changer ? peu probable on somme les vecteurs demi journée
                  pathClient = "../data/generation/dico_client.json",
                  pathProduit = "../data/generation/produit.json",
                  pathLienClientProduit = "../data/generation/lienClientProduit.json",
                  ) :
        
        """
        On genere l ensemble des paniers 
        necessite ecriturePaniers
               
        """
        self.pathClient = pathClient
        self.pathProduit = pathProduit
        self.pathLienClientProduit = pathLienClientProduit
        
        self.typePas = typePas
        self.typeFrequence = typeFrequence
        self.f_evenement = f_evenement
        
        self.dateDebutGeneral = dateDebutGeneral
        
        
        # on load l'ensemble des client dico par typeClient donne liste produit 
        f = open (pathClient , 'r')
        self.dico_client = json.loads(f.read ())
        f.close()
        
        #print (self.dico_client)
        
        # on calcule la date de fin de la generation
        # dans le cas, standart on simule sur une semaine
        horloge = gestionDate (typeFrequence, dateDebut = dateDebutGeneral  )
        self.dateFinControle = horloge.increment()
        self.liste_typeGen = horloge.liste_typeGen
        
        # on prepare ecriture panier avec ses paramètres
        self.E = ecriturePaniers (typePas,
                                  f_evenement = f_evenement ,
                                  pathProduit = pathProduit,)
        
        self.ecriturePanier = self.E.ecriturePanier
        
    def generation (self, nombreBoucle) :
        """
        appelle globale
        avec typePas on genere nombreBloucle fois l'ensemble des clients => ecriture panier
                    pathLienClientProduit = pathLienClientProduit,
                                  )
        self.ecriturePan si nombreBoucleEnCours > nombreBoucle on arrete la generation
          
        """
        # on commence à la date prevu toujours un lundi (coherence sur la semaine ?)
        horloge = gestionDate (self.typePas, dateDebut = self.dateDebutGeneral  )
        # verif de bon depart un lundi 
        jour = horloge.getJour ()
        if jour != 'lundi' :
            print ( "on commence pas le lundi" )
            raise ValueError
        dateDebutString = horloge.getDate()
        # on y va pour plus loin
        for nombre in range(0, nombreBoucle ) :
            self.ecritureClients (dateDebutString, )
            dateDebutString = horloge.increment ()
            continue
        
        return
            
            
        
        
    def ecritureClients (self,dateDebutString,
                                                                     
                        ) :
        """
        
        boucle sur l'ensemble des clients 
             format du dico :
                    { typeClient : [listeClients],
                    ....}
        
        
        
        """
        # on cherche la fin de la boucle pour generer par plage de frequence d'achat (mis dans client)
        horloge = gestionDate (self.typeFrequence, dateDebut = dateDebutString  )
        self.dateFinControle = horloge.increment()  # arret local de generation pour chaque client
        
        # on memorise l'ensemble des clients (dans ID client on a son type)
        listeTypeClients = [listeClient for listeClient in self.dico_client.keys()]
        listeClients = []
        for listeTypeClient in listeTypeClients :
            liste = self.dico_client [listeTypeClient]
            listeClients.extend(liste)
            continue
        
        # on boucle sur tous les clients   
        for client in listeClients :
            self.ecritureClient (client,
                                 dateDebutString,                          
                              )
            continue
        
        return len(listeClients)
            
        
    
        
        
        
    def ecritureClient (self, client,
                              dateDebutString,
                              isTest = False,
                              ) :
        """
        
        pour un client et
        
            pour les pas necessaire sur la periode qui defini pour generer les achats
                            sur la periode qui s'arrete fonction frqequence)
                            la date de fin de periode est calculee  self.dateFinControle
           
               -avons nous consommé le nombre de panier = frequence oui fin definitive ? (frequence dans client)
               -bon jour pour le client ? (ensemble des demi jour possible dans client)
               -quelle est la probalite de creer un panier pour ce client ?
                calculé frequence / len(list des demi jour possible)

                       si tout OK on genere le panier de client sur l'ensemble des pas
                       pour balayer les achats defini par la peridode 
                       qui permet de definir la frequence (frequence dans chaque client)
        
        
        """
        # on cree une horloge pour avancer sur la duree de la plage pour ce client
        # typePas calcule pour une date donne en fonction de typePas le nom voulu
        # pour savoir si ce client a le droit d'acheter
        # exemple typePas = 0 (demi_jour) ou typePas = 1 (jour)
        # client = [ID, isCouple, isCouple_enfant, comportement ]
        #  avec comportement = {temps, frequence}
        
        horlogeLocale = gestionDate ( self.typePas, dateDebut =  dateDebutString )
        ID = client [0]
        comportement = client [3]
        nombreDePanierMax = comportement ['frequence']
               
        # calcul de la proba de realisation d'un panier 
        
        #  liste de demi jour possible comme [lundi_matin, mardi_matin ]
        listePossible = comportement ['temps'] # liste des jours ou demi_jour .. possible pour ce client
        
        # nombreAchat par panier
        nombreMinAchat, nombreMaxAchat = comportement ['nombreAchat']
        
        # si le type frequence qui donne la plage d'apparition de la frequence voulu
        # est inferieur au pas d'avancement alors on fait une erreur de structure
        if self.typePas > self.typeFrequence :
            print (" erreur de structure la plage pour frequence est plus grande que le pas ?")
            raise ValueError
        
        # proba = frequence / nombre de demi_jour ou jour ..
        # decrit dans temps de ce client
        if len (listePossible) == 0 :
            # pas de generation car pas de demi jour possible
            return
        
        proba = float(nombreDePanierMax)/len(listePossible)
        if proba > 1.0 :
            print ("le nombre de panier max ne peut se faire sur la periode")
            print ("client =", client)
            raise ValueError
        
        memoireNombrePanier = 0  
        while (True ):
            # on boucle sur le nombre de pas pour atteindre self.dateFinControle
            # qui est la fin definit par le type de frequence
            #print (" minPanier =",memoireNombrePanier, "et max =", nombreDePanierMax)
            if memoireNombrePanier >= nombreDePanierMax :
                break
            # on regarde si le demi_jour ou le jour (ou la semaine ?) , ou le mois ou l'annee
            jour = horlogeLocale.getJour()
            if self.typePas == 0 : # toujours le cas actuellement (on travaille par demi jour)
                heure = horlogeLocale.getListeDate () [3]
                if heure < 12 :
                    jour = jour + "_matin"
                else:
                    jour = jour + "_apm"
            
                
            if jour not in listePossible :
                # c est pas le bon  jour pour le client on abandonne ce cas
                #print ("c est pas le bon  jour pour le client on abandonne ce cas")
                #print (jour, " et ", listePossible)
                date = horlogeLocale.increment()
                if date >  self.dateFinControle :
                    # on a fini la boucle sur pas 
                    break
                continue
                       
            # on tire la chance de faire un panier (fonction de proba)
            x = random.uniform(0.,1.)
            if isTest :
                print ("### proba ,x =",proba,", ",x, " dateControle =", self.dateFinControle )
            
            
            if x >= proba :
                #raté on boucle sur prochain pas si possible
                date = horlogeLocale.increment()
                if date >  self.dateFinControle :
                    # on a fini
                    break
                continue
            #print ("ecriture panier")
            # gagne on va creer le panier
            dateString = horlogeLocale.getDate()
            prixPanier = self.ecriturePanier (ID, dateString, nombreMinAchat, nombreMaxAchat
                                              )
            ########################################################################
            #print ("jour =",horlogeLocale.getJour())
            #print ("panier =",ID, dateString, nombreMinAchat, nombreMaxAchat)
            
            memoireNombrePanier += 1
            date = horlogeLocale.increment()
            if date >  self.dateFinControle :
                # on a fini
                break
            continue
        
        return

        

    
    
            
        
    
                                                                                                                                                                                                        


            
            
                 
                            
                 
            
            
        
        
        
        
        
        



    
    
    


In [7]:
pathClient = "../data/generation/dico_client.json"
# on load l'ensemble des client dico par typeClient donne liste produit 

f = open (pathClient , 'r')
dico_client = json.loads(f.read ())
f.close()

nombreClient = len(dico_client)



pathEvenements = "../data/generation/evenements1.txt"
isPrint = False
f_evenement = open (pathEvenements, 'w')
typePas = 0
typeFrequence = 2
dateDebutGeneral= '2021-02-01 00:00:00'

G = Generation  ( dateDebutGeneral,
                  f_evenement = f_evenement,
                  typePas = typePas, # demi journee (en general voir pls tard l'heure)
                  typeFrequence = typeFrequence, # semaine (en general voir plus tard si necessaire
                  # si nouveau contexte changer ? peu probable on somme les vecteurs demi journée
                  pathClient = "../data/generation/dico_client.json",
                  pathProduit = "../data/generation/produit.json",
                  pathLienClientProduit = "../data/generation/lienClientProduit.json",
                  )

nombreBoucle = 1000

G.generation (nombreBoucle)

f_evenement.close()
f = open(pathEvenements, 'r')
nombre = 0
while (True):
    li = f.readline()
    if li == "" :
        break
    nombre += 1
    continue
f.close()

if isPrint :
    print ("Avec generation(nombreBoucle = 2), nombre d'evenement generés ",nombre, " pour nombre clients", nombreClient)


print ("\nfin de generation pour nombreBoucle =", nombreBoucle)
print ("nombre d'evenement generés ",nombre, " pour nombre clients", nombreClient)



fin de generation pour nombreBoucle = 1000
nombre d'evenement generés  107131  pour nombre clients 12


In [4]:
print ("test ecriture evenement\n")
# pour memoire
liste_genre = ["homme",
        "femme",
         "couple",
         "couple_enfant",
         ]

liste_situation = ["cadre_sup",
            "cadre",
            "ouvrier"]

pathEvenements = "../data/generation/evenements.txt"

isPrint = False
f_evenement = open (pathEvenements, 'w')
E = ecritureEvenements (f_evenement=f_evenement,
                        isTest = False )
IDstring, dateString, produitChaine = ("tt1", "tt2", ['pp1', 'pp2'])
E.ecritureEvenement (IDstring, dateString, produitChaine, str (1.) )
E.ecritureEvenement (IDstring, dateString, produitChaine, str(2.) )
f_evenement.close()

f = open(pathEvenements, 'r')

l1 = f.readline()
if isPrint :
    print (l1)
l2 = f.readline()
if isPrint :
    print (l2)
l3 = f.readline()
if isPrint :
    print (l3)
f.close()

isPrint = False

print ("\n test panier 0") ###############################


isPrint = False
typePas = 0
f_evenement = open (pathEvenements, 'w')
P = ecriturePaniers (typePas,
                     f_evenement = f_evenement,
                     isTest = False )

ID = 'homme_cadre_sup_0'
dateString = '2021-01-15 00:00:00'
nombreMinAchat = 3
nombreMaxAchat = 5
for i in range (0, 2) :
    prixPanier = P.ecriturePanier (ID, dateString, nombreMinAchat, nombreMaxAchat )
    if isPrint :
        print ('\n\n\n################ essai # ',i, "pour un panier =",prixPanier)
    continue
f_evenement.close()

print ("\n test panier 1") ###############################


f_evenement = open (pathEvenements, 'w')


typePas = 0



P = ecriturePaniers (typePas,
                      f_evenement = f_evenement,
                      pathProduit = "../data/generation/produit.json",
                      pathLienClientProduit = "../data/generation/lienClientProduit.json",
                      isTest = False )

dateString = '2021-01-15 00:00:00'
nombreMinAchat = 1
nombreMaxAchat = 2
for genre in liste_genre :
    for situation in liste_situation :
        ID = '_'.join([genre, situation, '0'])
        prixPanier = P.ecriturePanier (ID, dateString,nombreMinAchat, nombreMaxAchat)
        if isPrint:
            print (" prix panier =", prixPanier)
        continue
        
f_evenement.close()

# on ecrit le resultat
f = open(pathEvenements, 'r')
l1 = f.read()
f.close()
if isPrint :
    print (l1)

isPrint = False

print ("\n\n##### test Ecriture par Clients")
pathEvenements = "../data/generation/evenements.txt"
typePas = 0 # demi_jour
typeFrequence = 2 # semaine 
dateDebutGeneral = "2021-02-01 00:00:00" # debutGeneral de la generation (all)
f_evenement = open (pathEvenements, 'w')
f_evenement.close()
# on verifie vide
f = open(pathEvenements, 'r')
l1 = f.read()
if len(l1) != 0 :
    print ("non vide ?",len(l1))

f_evenement = open (pathEvenements, 'w')


typePas = 0
typeFrequence = 2 # la semaine (0 demi jour, 1 jour, 2 semaine, 3 mois )

E = Generation (  dateDebutGeneral,
                  f_evenement = f_evenement,
                  typePas = typePas, # demi journee (en general voir pls tard l'heure)
                  typeFrequence = typeFrequence, # semaine (en general voir plus tard si necessaire
                  
                  # si nouveau contexte changer ? peu probable on somme les vecteurs demi journée
                  pathClient = "../data/generation/dico_client.json",
                  pathProduit = "../data/generation/produit.json",
                  pathLienClientProduit = "../data/generation/lienClientProduit.json",
                  )

client = ['couple_enfant_ouvrier_0', '', 'couple_enfant', {'temps': ['lundi_matin', 'lundi_apm',
                                                                     'mardi_matin', 'mardi_apm',
                                                                     'mercredi_matin', 'mercredi_apm',
                                                                     'jeudi_matin', 'jeudi_apm',
                                                                     'vendredi_matin', 'vendredi_apm'],
                                                           'frequence': 1,
                                                           'nombreAchat': [1,2]}]
                                                           

                                                           

dateDebutString = "2021-02-01 00:00:00" # debutGeneral de la generation (all)
horloge = gestionDate (typeFrequence, dateDebut = dateDebutString  )
E.dateFinControle = horloge.increment()  # arret local de generation pour ce client
#print ("fin de la semaine pour produire les paniers de ce client = ", E.dateFinControle)
for i in range (0,1000) : 
    E.ecritureClient ( client,
                       dateDebutString # date recurrente pour aller vers dateDebutGeneral
                       )
    continue
f_evenement.close()

print ()
# on ecrit le resultat
f = open(pathEvenements, 'r')
nombre = 0

while (True):
    li = f.readline()
    if li == "" :
        break
    nombre += 1
    continue
if isPrint :
    print ("nombre d'evenement generés",nombre)
    
    
print ("### test generation" )

isPrint = False
f_evenement = open (pathEvenements, 'w')
typePas = 0
typeFrequence = 2
dateDebutGeneral= '2021-01-15 00:00:00'

G = Generation  ( dateDebutGeneral,
                  f_evenement = f_evenement,
                  typePas = typePas, # demi journee (en general voir pls tard l'heure)
                  typeFrequence = typeFrequence, # semaine (en general voir plus tard si necessaire
                  # si nouveau contexte changer ? peu probable on somme les vecteurs demi journée
                  pathClient = "../data/generation/dico_client.json",
                  pathProduit = "../data/generation/produit.json",
                  pathLienClientProduit = "../data/generation/lienClientProduit.json",
                  )

dateDebutString = dateDebutGeneral

nombreClient = G.ecritureClients (dateDebutString,)

f_evenement.close()
f = open(pathEvenements, 'r')
nombre = 0
while (True):
    li = f.readline()
    if li == "" :
        break
    nombre += 1
    continue

if isPrint :
    print ("Pour ecritureClients nombre d'evenement generés =",nombre, " pour nombre clients", nombreClient)
    
#################################################### Generation complete
isPrint = False
f_evenement = open (pathEvenements, 'w')
typePas = 0
typeFrequence = 2
dateDebutGeneral= '2021-02-01 00:00:00'

G = Generation  ( dateDebutGeneral,
                  f_evenement = f_evenement,
                  typePas = typePas, # demi journee (en general voir pls tard l'heure)
                  typeFrequence = typeFrequence, # semaine (en general voir plus tard si necessaire
                  # si nouveau contexte changer ? peu probable on somme les vecteurs demi journée
                  pathClient = "../data/generation/dico_client.json",
                  pathProduit = "../data/generation/produit.json",
                  pathLienClientProduit = "../data/generation/lienClientProduit.json",
                  )

nombreBoucle = 1
G.generation (nombreBoucle)

f_evenement.close()
f = open(pathEvenements, 'r')
nombre = 0
while (True):
    li = f.readline()
    if li == "" :
        break
    nombre += 1
    continue
f.close()

if isPrint :
    print ("Avec generation(nombreBoucle = 1) nombre d'evenement generés ",nombre, " pour nombre clients", nombreClient)

pathEvenements = "../data/generation/evenements1.txt"
isPrint = False
f_evenement = open (pathEvenements, 'w')
typePas = 0
typeFrequence = 2
dateDebutGeneral= '2021-02-01 00:00:00'

G = Generation  ( dateDebutGeneral,
                  f_evenement = f_evenement,
                  typePas = typePas, # demi journee (en general voir pls tard l'heure)
                  typeFrequence = typeFrequence, # semaine (en general voir plus tard si necessaire
                  # si nouveau contexte changer ? peu probable on somme les vecteurs demi journée
                  pathClient = "../data/generation/dico_client.json",
                  pathProduit = "../data/generation/produit.json",
                  pathLienClientProduit = "../data/generation/lienClientProduit.json",
                  )

nombreBoucle = 2
G.generation (nombreBoucle)

f_evenement.close()
f = open(pathEvenements, 'r')
nombre = 0
while (True):
    li = f.readline()
    if li == "" :
        break
    nombre += 1
    continue
f.close()

if isPrint :
    print ("Avec generation(nombreBoucle = 2), nombre d'evenement generés ",nombre, " pour nombre clients", nombreClient)


print ("\nfin de generation pour nombreBoucle =", nombreBoucle)
print ("nombre d'evenement generés ",nombre, " pour nombre clients", nombreClient)


    











            
            
            
            
                   
            
                
                
            
            
            
            

test ecriture evenement


 test panier 0

 test panier 1


##### test Ecriture par Clients

### test generation

fin de generation pour nombreBoucle = 2
nombre d'evenement generés  192  pour nombre clients 24


In [19]:
from time import time
class somme () :
    def __init__ (self) :
        self.resultat = 0
    def somme (self, i) :
        self.resultat += i
        return
t = time ()
S = somme ()       
operateur = S.somme        
    
[operateur (i) for i in range(1, 10000)]
duree = time() -t

duree


0.002885580062866211

In [ ]:
from time import time
class somme () :
    def __init__ (self) :
        self.resultat = 0
    def somme (self, i) :
        self.resultat += i
        return
t = time ()
S = somme ()       
operateur = S.somme 
resultat = 0
for i in range (1, 10000) :
    resultat += i
    continue
    

duree = time() -t

duree

In [ ]:
nombre = 0
proba = 3./5

for _ in range(0, 5) :
    x = random.random ()
    if x >= proba :
        nombre += 1
        
nombre
    
    